**News Category Classification**

In [ ]:
!pip install feedparser
import feedparser
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

def veri_yukle():
    veri = pd.read_excel('news_category.xlsx')
    metinler = veri['HABERLER'].values
    etiketler = veri['ETIKET'].values
    return veri, metinler, etiketler

def veri_tokenize(veri):
    veri['Cumle'] = veri['HABERLER'].apply(lambda x: sent_tokenize(str(x)))
    veri['Kelime'] = veri['HABERLER'].apply(lambda x: word_tokenize(str(x)))
    return veri

def stopwords_filtrele(veri):
    stopwords_listesi = stopwords.words("turkish")
    filtrelenmis_kelimeler = [
        [kelime for kelime in kelime_listesi if kelime.lower() not in stopwords_listesi]
        for kelime_listesi in veri['Kelime']
    ]
    return filtrelenmis_kelimeler

def word2vec_egit(filtrelenmis_kelimeler):
    word2vec_modeli = Word2Vec(sentences=filtrelenmis_kelimeler, vector_size=100, window=5, min_count=1, workers=4)

    with open('word2vec_model.py', 'wb') as dosya:
        pickle.dump(word2vec_modeli, dosya)
    return word2vec_modeli

def veri_onisleme(X, y):
    etiket_encoder = LabelEncoder()
    y = etiket_encoder.fit_transform(y)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    X = tokenizer.texts_to_matrix(X, mode='binary')
    return X, y

def model_olustur_egit(X_egitim, y_egitim, X_test, y_test):
    model = Sequential([
      Dense(256, input_shape=(X_egitim.shape[1],), activation='relu'),
      Dense(128, activation='relu'),
      Dropout(0.3),
      Dense(64, activation='relu'),
      Dense(32, activation='relu'),
      Dropout(0.2),
      Dense(len(set(y_egitim)), activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    modelsekil = model.fit(X_egitim, y_egitim, epochs=15, batch_size=32, validation_data=(X_test, y_test))
    return model

def modeli_degerlendir(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

def main():
    veri, X, y = veri_yukle()
    veri = veri_tokenize(veri)
    filtrelenmis_kelimeler = stopwords_filtrele(veri)
    word2vec_model = word2vec_egit(filtrelenmis_kelimeler)
    X, y = veri_onisleme(X, y)
    X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = model_olustur_egit(X_egitim, y_egitim, X_test, y_test)
    modeli_degerlendir(model, X_test, y_test)

if __name__ == "__main__":
    main()
    
feed_url = "https://www.cnnturk.com/feed/rss/all/news"
parsed_feed = feedparser.parse(feed_url)
haberler = [entry.title for entry in parsed_feed.entries]
haberler_encoded = tokenizer.texts_to_matrix(haberler, mode='binary')

olasiliklar = model.predict(haberler_encoded)
etiket_etiketleri = np.argmax(olasiliklar, axis=1)
etiket = etiket_encoder.inverse_transform(etiket_etiketleri)

df = pd.DataFrame({'HABERLER': haberler, 'ETIKET': etiket})
df.to_csv('HABERLER.csv', index=False)

from google.colab import files
files.download('HABERLER.csv')

from keras.utils import plot_model

#Ağ Şekli
model = Sequential([
    Dense(256, input_shape=(X_egitim.shape[1],), activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(len(set(y_egitim)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# Eğitim sırasında kayıp ve doğruluk metriklerini görselleştirme
plt.figure(figsize=(12, 4))

# Eğitim ve doğrulama kaybı
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epochs')
plt.ylabel('Kayıp')
plt.legend()

# Eğitim ve doğrulama doğruluğu
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epochs')
plt.ylabel('Doğruluk')
plt.legend()

plt.tight_layout()
plt.show()

from sklearn.metrics import confusion_matrix

# Test verileri üzerinde modeli değerlendirme
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Hata matrisi oluşturma
cm = confusion_matrix(y_test, predicted_classes)

# Hata matrisini görselleştirme
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square=True, cmap='Blues')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Öngörülen Etiket')
all_sample_title = 'Sınıflandırma Raporu'
plt.title(all_sample_title, size = 15)
plt.show()

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

# Değerlendirme Metriklerinin Hesaplanması
y_tahmin = np.argmax(model.predict(X_test), axis=-1)

# Hassasiyet
hassasiyet = precision_score(y_test, y_tahmin, average='weighted')
print(f'Hassasiyet: {hassasiyet}')

# Duyarlılık
duyarlılık = recall_score(y_test, y_tahmin, average='weighted')
print(f'Duyarlılık: {duyarlılık}')

# F1 Skoru
f1 = f1_score(y_test, y_tahmin, average='weighted')
print(f'F1 Skoru: {f1}')

# Eğitilmiş modeli kullanarak olasılıkları hesaplama
olasiliklar = model.predict(X_test)

# Çok sınıflı ROC eğrileri için etiketleri tek seferde kodlama
y_test_one_hot = label_binarize(y_test, classes=np.unique(y_test))

# Her sınıf için ROC eğrisini ve ROC alanını hesaplama
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(np.unique(y_test))):
    fpr[i], tpr[i], _ = roc_curve(y_test_one_hot[:, i], olasiliklar[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Her sınıf için ROC eğrilerini çizdirme
plt.figure(figsize=(8, 6))
for i in range(len(np.unique(y_test))):
    plt.plot(fpr[i], tpr[i], label=f'ROC Eğrisi (Sınıf {i}) (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], linestyle='--', color='red')
plt.xlabel('Yanlış Pozitif Oranı')
plt.ylabel('Doğru Pozitif Oranı')
plt.title('ROC Eğrileri - Çoklu Sınıf')
plt.legend()
plt.show()

**Sentiment Analysis**

In [ ]:
!pip install feedparser
import feedparser
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

def veri_yukle():
    veri = pd.read_csv('sentiment_analysis.csv')
    X = veri['text'].values
    y = veri['label'].values
    return veri, X, y

def cumleleri_tokenlestir(veri):
    veri['CUMLE'] = veri['text'].apply(lambda x: sent_tokenize(str(x)))
    veri['KELIME'] = veri['text'].apply(lambda x: word_tokenize(str(x)))
    return veri

def stopwords_filtrele(veri):
    stopwords_listesi = set(stopwords.words("turkish"))
    veri['FILTRELI_KELIME'] = veri['KELIME'].apply(lambda x: [kelime for kelime in x if kelime.lower() not in stopwords_listesi])
    return veri

def word2vec_egit(filtered_words):
    word2vec_modeli = Word2Vec(sentences=filtered_words, vector_size=100, window=5, min_count=1, workers=4)

    with open('word2vec_model.py', 'wb') as dosya:
        pickle.dump(word2vec_modeli, dosya)
    return word2vec_modeli

def veriyi_onisle(X, y):
    etiket_encoder = LabelEncoder()
    y = etiket_encoder.fit_transform(y)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    X = tokenizer.texts_to_matrix(X, mode='binary')
    return X, y

def model_olustur(X_egitim, y_egitim, X_test, y_test):
    model = Sequential([
        Dense(64, input_shape=(X_egitim.shape[1],), activation='relu'),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dropout(0.2),
        Dense(len(set(y_egitim)), activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_egitim, y_egitim, epochs=15, batch_size=32, validation_data=(X_test, y_test))
    return model

def modeli_degerlendir(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

def main():
    data, X, y = veri_yukle()
    data = cumleleri_tokenlestir(data)
    filtered_words = stopwords_filtrele(data)
    word2vec_model = word2vec_egit(filtered_words)
    X, y = veriyi_onisle(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = model_olustur(X_train, y_train, X_test, y_test)
    modeli_degerlendir(model, X_test, y_test)

if __name__ == "__main__":
    main()

from tensorflow.keras.preprocessing.sequence import pad_sequences

url = 'https://www.haberler.com/ekonomi/turk-is-ten-hukumete-asgari-ucret-resti-aclik-16605261-haberi/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
yorum_etiketleri = soup.find_all(['div'], class_=['hbcRow', 'hbcInRow'])

yorum_listesi = []
for yorum_etiketi in yorum_etiketleri:
    yorum_listesi.append(yorum_etiketi.text.strip())

tokenizer = Tokenizer()
tokenizer.fit_on_texts(yorum_listesi)
yorumlar_encoded = tokenizer.texts_to_matrix(yorum_listesi, mode='binary')

kelime_vektorleri_yorum = []
for yorum in yorum_listesi:
    kelimeler = word_tokenize(yorum)
    vektor = np.zeros(100)
    sayac = 0
    for kelime in kelimeler:
        if kelime in word2vec_modeli.wv:
            vektor += word2vec_modeli.wv[kelime]
            sayac += 1
    if sayac != 0:
        vektor /= sayac
    kelime_vektorleri_yorum.append(vektor)

padded_sequences = pad_sequences(kelime_vektorleri_yorum, maxlen=100, padding='post', truncating='post')
yorumlar_encoded = np.array(padded_sequences)

tahmin_edilen_olasiliklar = model.predict(yorumlar_encoded)
tahmin_edilen_etiketler = np.argmax(tahmin_edilen_olasiliklar, axis=1)
tahmin_edilen_kategoriler = etiket_encoder.inverse_transform(tahmin_edilen_etiketler)


tahmin_edilen_veri = pd.DataFrame({'text': yorum_listesi, 'label': tahmin_edilen_kategoriler})
tahmin_edilen_veri.to_csv('Tahmin_Edilen_Yorumlar.csv', index=False, encoding='utf-8')

#Ağ Şekli
model = Sequential([
        Dense(64, input_shape=(X_egitim.shape[1],), activation='relu'),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dropout(0.2),
        Dense(len(set(y_egitim)), activation='softmax')
    ])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# Eğitim sırasında kayıp ve doğruluk metriklerini görselleştirme
plt.figure(figsize=(12, 4))

# Eğitim ve doğrulama kaybı
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epochs')
plt.ylabel('Kayıp')
plt.legend()

# Eğitim ve doğrulama doğruluğu
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epochs')
plt.ylabel('Doğruluk')
plt.legend()

plt.tight_layout()
plt.show()

from sklearn.metrics import confusion_matrix

# Test verileri üzerinde modeli değerlendirme
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)


# Hata matrisi oluşturma
cm = confusion_matrix(y_test, predicted_classes)

# Hata matrisini görselleştirme
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square=True, cmap='Blues')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Öngörülen Etiket')
all_sample_title = 'Sınıflandırma Raporu'
plt.title(all_sample_title, size = 15)
plt.show()

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

# Değerlendirme Metriklerinin Hesaplanması
y_tahmin = np.argmax(model.predict(X_test), axis=-1)

# Hassasiyet
hassasiyet = precision_score(y_test, y_tahmin, average='weighted')
print(f'Hassasiyet: {hassasiyet}')

# Duyarlılık
duyarlılık = recall_score(y_test, y_tahmin, average='weighted')
print(f'Duyarlılık: {duyarlılık}')

# F1 Skoru
f1 = f1_score(y_test, y_tahmin, average='weighted')
print(f'F1 Skoru: {f1}')

# Eğitilmiş modeli kullanarak olasılıkları hesaplama
olasiliklar = model.predict(X_test)

# Çok sınıflı ROC eğrileri için etiketleri tek seferde kodlama
y_test_one_hot = label_binarize(y_test, classes=np.unique(y_test))

# Her sınıf için ROC eğrisini ve ROC alanını hesaplama
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(np.unique(y_test))):
    fpr[i], tpr[i], _ = roc_curve(y_test_one_hot[:, i], olasiliklar[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Her sınıf için ROC eğrilerini çizdirme
plt.figure(figsize=(8, 6))
for i in range(len(np.unique(y_test))):
    plt.plot(fpr[i], tpr[i], label=f'ROC Eğrisi (Sınıf {i}) (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], linestyle='--', color='red')
plt.xlabel('Yanlış Pozitif Oranı')
plt.ylabel('Doğru Pozitif Oranı')
plt.title('ROC Eğrileri - Çoklu Sınıf')
plt.legend()
plt.show()